In [1]:
from __future__ import annotations
import cirq
import numpy as np
import scipy

In [2]:
class ParametricCircuit:
    def __init__(self, qubit_circuit_length: int, rotation_parameters: list):
        """

        :param qubit_circuit_length: amount of qubits
        :param rotation_parameters: list of 2 * qubit_circuit_length rotation angles
        """
        self.qubit_circuit_length = qubit_circuit_length
        self.rotation_parameters = rotation_parameters

    @property
    def generate_circuit(self) -> cirq.Circuit:
        """

        :return: cirq.Circuit object
        """
        qubit_circuit = cirq.LineQubit.range(self.qubit_circuit_length)
        new_circuit = cirq.Circuit()
        for i in range(len(qubit_circuit)):
            new_circuit.append(cirq.Rx(rads=self.rotation_parameters[i])(qubit_circuit[i]))
        for i in range(len(qubit_circuit) - 1):
            new_circuit.append(cirq.CNOT(qubit_circuit[i], qubit_circuit[i + 1]))
        for i in range(len(qubit_circuit)):
            new_circuit.append(cirq.Rx(rads=self.rotation_parameters[i + len(qubit_circuit)])(qubit_circuit[i]))
        for i in range(len(qubit_circuit) - 1):
            new_circuit.append(cirq.CNOT(qubit_circuit[i], qubit_circuit[i + 1]))
        return new_circuit

    @property
    def probability_vector(self) -> numpy.ndarray:
        """

        :return: the list of probability to collapse in |0> state after z-basic measure for each qubit
        """
        simulator = cirq.Simulator()
        result = simulator.simulate(self.generate_circuit)
        probability_list = []
        for i in range(self.qubit_circuit_length):
            probability_list.append(1 / 2 * (1 + result.bloch_vector_of(cirq.LineQubit(i))[2]))
        probability_list = np.array(probability_list)
        return probability_list

In [4]:
def cost_function(probability_vector: np.ndarray) -> float:
    """

    :param probability_vector: vector of probabilities for 5 qubits
    :return: the cost value
    """
    bilinear_form = np.array([[-3, 2, 2, 2, 2],
                              [0, -3, 2, 2, 2],
                              [0, 0, -3, 2, 2],
                              [0, 0, 0, -3, 2],
                              [0, 0, 0, 0, -3]])
    return probability_vector @ bilinear_form @ probability_vector + 4


def cost_function_of_angular_argument(rotation_parameters: list) -> float:
    """

    :param rotation_parameters: angles of rotation for 5 qubits
    :return: the cost value (using class ParametricCircuit)
    """
    return cost_function(ParametricCircuit(5, rotation_parameters).probability_vector)

In [3]:
def minimize_cost_function() -> list:
    """

    :return: result of applying function from scipy.optimize to the cost function
    """
    starting_point = np.zeros(10)
    lower_bound = -2 * np.pi * np.ones(10)
    upper_bound = -lower_bound
    bounds = [(low, high) for low, high in zip(lower_bound, upper_bound)]
    # minimizer_kwargs = dict(bounds=bounds)
    description = scipy.optimize.dual_annealing(cost_function_of_angular_argument, bounds=bounds)
    return description

In [5]:
print(minimize_cost_function())

     fun: 1.585482516208714e-05
 message: ['Maximum number of iteration reached']
    nfev: 23862
    nhev: 0
     nit: 1000
    njev: 351
  status: 0
 success: True
       x: array([-3.57580413e-04,  3.81886309e-04, -2.61748442e-04,  3.13965633e+00,
       -5.81623359e+00, -6.28237153e+00, -6.28275711e+00,  3.14259389e+00,
        3.14265221e+00, -3.61018255e+00])


In [6]:
print(minimize_cost_function())

     fun: 4.6133802418246717e-05
 message: ['Maximum number of iteration reached']
    nfev: 22751
    nhev: 0
     nit: 1000
    njev: 250
  status: 0
 success: True
       x: array([-3.14072845e+00,  3.14151466e+00, -6.28065016e+00, -3.13902456e+00,
        4.10760625e+00,  3.14099015e+00,  3.14133944e+00, -2.99540280e-04,
       -3.14379355e+00,  2.17229746e+00])


In [7]:
print(minimize_cost_function())

     fun: 4.792197504954743e-05
 message: ['Maximum number of iteration reached']
    nfev: 21640
    nhev: 0
     nit: 1000
    njev: 149
  status: 0
 success: True
       x: array([ 6.67036483e-05,  3.41892297e-04,  6.28197140e+00, -3.14085414e+00,
        2.58517926e+00, -6.28194058e+00, -3.13955820e+00,  6.28282480e+00,
       -3.14476325e+00, -2.58431094e+00])


In [8]:
print(minimize_cost_function())

     fun: 5.674344299677614e-05
 message: ['Maximum number of iteration reached']
    nfev: 21970
    nhev: 0
     nit: 1000
    njev: 179
  status: 0
 success: True
       x: array([ 3.13920898e+00,  3.14382496e+00,  1.86307417e-03, -3.14105581e+00,
        3.14929029e+00, -6.28267196e+00,  6.28135242e+00,  3.14132909e+00,
        2.01265147e-03,  3.13410067e+00])


In [9]:
print(minimize_cost_function())

     fun: 3.290168236347313e-05
 message: ['Maximum number of iteration reached']
    nfev: 22245
    nhev: 0
     nit: 1000
    njev: 204
  status: 0
 success: True
       x: array([-1.54836836e-03, -3.13984409e+00,  6.28238274e+00, -3.14194190e+00,
        3.50678386e+00,  6.28271255e+00,  1.32538250e-04,  3.14175012e+00,
        3.14382590e+00, -3.68113552e-01])


In [10]:
print(minimize_cost_function())

     fun: 3.194803405648372e-05
 message: ['Maximum number of iteration reached']
    nfev: 22058
    nhev: 0
     nit: 1000
    njev: 187
  status: 0
 success: True
       x: array([ 3.14275708, -3.14173256, -6.28131527, -3.14055755,  5.27406771,
       -3.14338965, -3.14073819,  3.14159923,  6.28256345, -2.13289843])


In [11]:
print(minimize_cost_function())

     fun: 2.6702834315273094e-05
 message: ['Maximum number of iteration reached']
    nfev: 21563
    nhev: 0
     nit: 1000
    njev: 142
  status: 0
 success: True
       x: array([-1.66389441e-03,  6.28307020e+00,  3.14048875e+00,  3.14021834e+00,
        6.12555405e+00, -4.84094702e-04,  3.14222804e+00, -3.14326316e+00,
        1.03071912e-03, -2.98332185e+00])


In [12]:
print(minimize_cost_function())

     fun: 2.3245784366743294e-05
 message: ['Maximum number of iteration reached']
    nfev: 22377
    nhev: 0
     nit: 1000
    njev: 216
  status: 0
 success: True
       x: array([-6.28312708e+00,  6.28314616e+00, -3.14163771e+00,  3.14268926e+00,
       -5.76547195e+00,  1.76502744e-03, -3.14213569e+00,  3.14263043e+00,
        3.14138280e+00, -3.65982090e+00])


In [13]:
print(minimize_cost_function())

     fun: 3.623955139175905e-05
 message: ['Maximum number of iteration reached']
    nfev: 22795
    nhev: 0
     nit: 1000
    njev: 254
  status: 0
 success: True
       x: array([ 3.14229279e+00, -1.65014515e-04, -3.13997772e+00, -6.28294011e+00,
        2.80333813e+00, -3.14093475e+00,  1.84002922e-03, -1.03415116e-03,
       -3.14300448e+00,  3.35799619e-01])
